En esta primmera linea se cargan los modulos necesarios para usar en algoritmo.

In [2]:
from google.colab import files
files.upload()

Saving __init__.py to __init__.py
Saving AlgorithmMod.py to AlgorithmMod.py
Saving Math.py to Math.py
Saving Model.py to Model.py
Saving OmnFitness.py to OmnFitness.py
Saving Simulate.py to Simulate.py
Saving Training.txt to Training.txt


{'__init__.py': b'',
 'AlgorithmMod.py': b'import time\r\nimport random\r\nimport tracemalloc\r\nimport sys\r\nimport os\r\nimport pickle\r\nfrom deap import tools\r\nSCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))\r\nsys.path.append(os.path.dirname(SCRIPT_DIR))\r\nsys.path.insert(0, \'~/libs\')\r\nsys.path.insert(1, \'~/Simulation\')\r\ndef update_progress(job_title, progress):\r\n    try:\r\n        os.system(\'clc\')\r\n    except:\r\n        os.system(\'clear\')\r\n    finally:\r\n        os.system(\'cls\')\r\n    length = 20 # modify this to change the length\r\n    block = int(round(length*progress))\r\n    msg = "\\r{0}: [{1}] {2}%".format(job_title, "#"*block + "-"*(length-block), round(progress*100, 2))\r\n    if progress >= 1: msg += " DONE\\r\\n"\r\n    sys.stdout.write(msg)\r\n    sys.stdout.flush()\r\n\r\ndef update_msg(job, prog):\r\n    length2 = 20 # modify this to change the length\r\n    block2 = int(round(length2*prog))\r\n    msg2 = "\\n{0}: [{1}] {2}%".form

Cada vez que se va a realizar una corrida se debe instalar la libreria DEAP

In [4]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.2 MB/s eta 0:00:00


Programa principal evolutivo, aqui se definen las funciones, terminales y función aptitud del algoritmo evolutivo

In [ ]:
#!/usr/bin/env python3
import os, sys
SCRIPT_DIR = os.path.dirname(os.path.abspath('Gp-2Robots'))
sys.path.append(os.path.dirname(SCRIPT_DIR))

from Math import Plus, Minus, Divide, Times,Power, Maximum, Minimum, Atan2r
from Math import Sqrt, Sqr, Logn, Exp, Sinh, Cosh, Tanh, Csch, Sech, Coth, Cos, Sin, Tan, Csc, Sec, Cot, Asin, Acos, Norm, Abs, Real, Atanr
import AlgorithmMod
import OmnFitness
from Simulate import Evasion2Robots
#No-Downlodable
import gc
import os
import csv
import time
import math
import socket
import itertools
import traceback
import multiprocessing as MP
#Downlodable
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from deap import tools
from deap import gp
from deap import creator
from deap import base
N=2
def Main(N,path):
    CPU_count = MP.cpu_count()-1
    start = time.time()
    #   Algoritmo de Programacion Genetica para evolucionar
    #   un modelo de navegación de un robot movil.
    #   Version: 1.14.17092024
    #   Autor: MSc. Rodrigo A. Villalvazo Covian
    #   Posgrado de Control y Robótica
    #   ITE, Ensenada B.C. Mexico
    """
    Program and Problem parameters
    """
    #Tiempo de llegada
    td=4.0
    #numero de generaciones
    Ngen=100#increase Factor
    #numero de poblacion
    Npop=300#Decrease factor
    #numero de condiciones iniciales disponibles
    Ncon=10
    #condicionador, el factor debe ser menor al numero de condiciones
    #Factor=Factorial*round(Ngen*((abs(((CPU_count*Ncon)*0.25)-Ncon))/Npop))#Factor de condicion para procesos respecto a las condiciones disponibles
    Proc=2
    #Numero de corrida
    run=int(N)
    #Participantes en el torneo
    winners=7
    def MultProcess(ind,td,Npros,Ncon):
      firstTime=time.time()
      fit=[]
      for i in range (Ncon):
          ret_fit1=0
          ret_fit2=0
          ret_fit3=0
          FileD=open('Training.txt')#Direccion contenedora del archivo de condiciones iniciales
          #td=4.0
          index=i
          #ind='Power(Abs(Sec(xd2)), tR)'
          ind='Sin(Vy1)'
          #ind='Logn(Sqrt(Norm(xdp2)))'
          reader=csv.reader(FileD, delimiter=' ',skipinitialspace=True)#Leemos todo el documento
          lineData = list()#variable para las lineas
          cols=next(reader)#variable para las columnas
          #  Leemos las columnas
          for col in cols:
              lineData.append(list())
          #Leemos las lineas
          for line in reader:
              for j in range(0,len(lineData)):
                  lineData[j].append(line[j])
          #Guardamos el resultado como un diccionario
          data=dict()
          for j in range(0, len(cols)):
              data[j]=lineData[j]
          Cd=[0.5,0.5]
          #Hacemos un barrido de las condiciones inciales y accedemos a las que necesitemos
          Rob1=[lineData[0][index],lineData[1][index]]#Posicion inicial del robot 1
          Rob2=[lineData[2][index],lineData[3][index]]#Posicion inicial del robot 2
          td=float(lineData[6][index])
          tf=td+0.50
          rRob1,rRob2=0.02,0.02#Radio del robot 1 y robot2
          s,Desires=Evasion2Robots(ind,tf,td,FileD,Rob1,Rob2,rRob1,rRob2,Cd)#Simulacion de los robots para cada condicion inicial y cada individuo
          radios=Desires[0]
          pd=Desires[1]
          PdR1=pd[0]
          PdR2=pd[1]
          #Tiempo que tardo en llegar el robot
          timeSavedR1=s[0]
          timeSavedR2=s[1]
          #Variable temporal almacendada para cada robot
          timeR1=s[5]#Variable t del robot 1
          timeR2=s[19]#Variable t del robot 2
          Try1=s[2]#Posicion en el eje X del robot 1
          Try2=s[10]#Posicion en el eje Y del robot 1
          Try3=s[11]#Posicion en el eje X del robot 2
          Try4=s[12]#Posicion en el eje Y del robot 2
          dsR1=s[20]#Distancia entre cada robot visto desde el robot 1
          dsR2=s[21]#Distancia entre cada robot visto desde el robot 2
          n1=np.size(timeR1)-1#Variable auxiliar
          n2=np.size(timeR2)-1#Variable auxiliar

          TryR1=[Try1[n1],Try2[n1]]#Vector posicion del robot 1
          TryR2=[Try3[n2],Try4[n2]]#Vector posicion del robot 2

          ErrorPosRob1=np.linalg.norm([TryR1[0]-PdR1[0],TryR1[1]-PdR1[1]])#Error de posicion del robot 1
          ErrorPosRob2=np.linalg.norm([TryR2[0]-PdR2[0],TryR2[1]-PdR2[1]])#Error de posicion del robot 2
          ErrorTiempoR1=abs(timeSavedR1-td)#Error del tiempo de llegada del robot 1 y el tiempo limite
          ErrorTiempoR2=abs(timeSavedR2-td)#Error del tiempo de llegada del robot 2 y el tiempo limite
          radio=radios[0]+radios[1]
          tolerance=0.3
          Ext1=(timeSavedR1==0)and(ErrorTiempoR1>tolerance)
          Ext2=(timeSavedR2==0)and(ErrorTiempoR2>tolerance)

          old_fitness=(ErrorPosRob1**2)+(ErrorPosRob2**2)+(ErrorTiempoR1**2)+(ErrorTiempoR2**2)#Fitness error cuadratico medio
          if (math.isnan(old_fitness) or math.isinf(old_fitness)):#Penalizamos si el fitness es infinito o si no es un numero
              ret_fit1=old_fitness+120.0
          elif((dsR1<=radio)and(dsR2<=radio)):
              ret_fit2=old_fitness+100.0
          elif(Ext1 or Ext2):#Penalizamos la diferencia entre tiempos de llegada de cada robot
              ret_fit3=old_fitness+110.0
          else:
              old_fitness=old_fitness
          new_fitness=ret_fit1+ret_fit2+ret_fit3+old_fitness#Fitness error cuadratico medio
          fit.append(new_fitness)
      MYFitness=np.mean(fit)
      return MYFitness,



    toolbox = base.Toolbox()
    pset = gp.PrimitiveSetTyped("MAIN",itertools.repeat(float,26),float)
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,pset=pset)
    # Attribute generator
    toolbox.register("expr_init", gp.genHalfAndHalf, pset=pset, min_=1, max_=11)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate",MultProcess,td=td,Npros=Proc,Ncon=Ncon)
    toolbox.register("select", tools.selTournament, tournsize=winners)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("expr_mut", gp.genFull, min_=1, max_=11)
    toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
    #Parametros
    pset.addPrimitive(Plus,[float,float],float)#Suma normalizada
    pset.addPrimitive(Minus,[float,float],float)#Resta normalizada
    pset.addPrimitive(Divide,[float,float],float)#Division Real normalizada
    pset.addPrimitive(Times,[float,float],float)#Multiplicacion normalizada
    pset.addPrimitive(Power,[float,float],float)#Exponenciacion
    pset.addPrimitive(Maximum,[float,float],float)#Valor maximo
    pset.addPrimitive(Minimum,[float,float],float)#Valor minimo
    pset.addPrimitive(Atan2r,[float,float],float)#arcotangente real de 2 valores normalizada
    #Arity 1
    pset.addPrimitive(Sqrt,[float],float)#Raiz cuadrada
    pset.addPrimitive(Sqr,[float],float)#Cuadrado de un escalar
    pset.addPrimitive(Logn,[float],float)#Logaritmo
    pset.addPrimitive(Exp,[float],float)#Exponencial
    pset.addPrimitive(Sinh,[float],float)#Seno hyperbolico
    pset.addPrimitive(Cosh,[float],float)#Coseno hyperbolico
    pset.addPrimitive(Tanh,[float],float)#Tangente hyperbolica
    pset.addPrimitive(Csch,[float],float)#Cosecante hyperbolico
    pset.addPrimitive(Sech,[float],float)#Secante hyperobolica
    pset.addPrimitive(Coth,[float],float)#Cotangente hyperbolico
    pset.addPrimitive(Cos,[float],float)#coseno
    pset.addPrimitive(Sin,[float],float)#seno
    pset.addPrimitive(Tan,[float],float)#tangente
    pset.addPrimitive(Csc,[float],float)#cosecante
    pset.addPrimitive(Sec,[float],float)#secante
    pset.addPrimitive(Cot,[float],float)#cotangente
    pset.addPrimitive(Asin,[float],float)#Arcoseno
    pset.addPrimitive(Acos,[float],float)#Arcocoseno
    pset.addPrimitive(Norm,[float],float)#norma
    pset.addPrimitive(Abs,[float],float)#absoluto
    pset.addPrimitive(Real,[float],float)#real
    pset.addPrimitive(Atanr,[float],float)#arcotangente real
    #Terminales
    pset.addTerminal("qx2",str)#0
    pset.addTerminal("qy2",str)#1
    pset.addTerminal("qx1",str)#2
    pset.addTerminal("qy1",str)#3
    pset.addTerminal("xd1",str)#4
    pset.addTerminal("xdp1",str)#5
    pset.addTerminal("yd1",str)#6
    pset.addTerminal("ydp1",str)#7
    pset.addTerminal("xd2",str)#8
    pset.addTerminal("xdp2",str)#9
    pset.addTerminal("yd2",str)#10
    pset.addTerminal("ydp2",str)#11
    pset.addTerminal("Vx1",str)#12
    pset.addTerminal("Vy1",str)#13
    pset.addTerminal("Vx2",str)#14
    pset.addTerminal("Vy2",str)#15
    pset.addTerminal("t1",str)#16
    pset.addTerminal("t2",str)#17
    pset.addTerminal("td",str)#18
    pset.addTerminal("q1",str)#21
    pset.addTerminal("q2",str)#22
    pset.addTerminal("qd1",str)#23
    pset.addTerminal("qd2",str)#24
    pset.addTerminal("V1",str)#25
    pset.addTerminal("V2",str)#26
    pset.addTerminal("tR",str)#27
    pset.addTerminal("tdR",str)#28
    #pset.addTerminal("OrR1R1",str)#19
    #pset.addTerminal("OrR2R1",str)#20
    """Rename the Args to our variables names"""
    pset.renameArguments(ARG0='qx2')
    pset.renameArguments(ARG1='qy2')
    pset.renameArguments(ARG2='qx1')
    pset.renameArguments(ARG3='qy1')
    pset.renameArguments(ARG4='xd1')
    pset.renameArguments(ARG5='xdp1')
    pset.renameArguments(ARG6='yd1')
    pset.renameArguments(ARG7='ydp1')
    pset.renameArguments(ARG8='xd2')
    pset.renameArguments(ARG9='xdp2')
    pset.renameArguments(ARG10='yd2')
    pset.renameArguments(ARG11='ydp2')
    pset.renameArguments(ARG12='Vx1')
    pset.renameArguments(ARG13='Vy1')
    pset.renameArguments(ARG14='Vx2')
    pset.renameArguments(ARG15='Vy2')
    pset.renameArguments(ARG16='t1')
    pset.renameArguments(ARG17='t2')
    pset.renameArguments(ARG18='td')
    pset.renameArguments(ARG19='q1')
    pset.renameArguments(ARG20='q2')
    pset.renameArguments(ARG21='qd1')
    pset.renameArguments(ARG22='qd2')
    pset.renameArguments(ARG23='V1')
    pset.renameArguments(ARG24='V2')
    pset.renameArguments(ARG25='tR')
    pset.renameArguments(ARG26='tdR')
    #pset.renameArguments(ARG19='OrR1R1')
    #pset.renameArguments(ARG20='OrR2R1')
    """Get the indiviual expresion"""
    #expr = toolbox.individual()
    #del pset
    gc.collect()
    #Estadisticas
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)
    pop = toolbox.population(n=Npop)
    hof = tools.HallOfFame(1)
    chk_name=str('checkpoint'+str(run)+'.pkl')
    newCheckP=path+chk_name
    CheckP=AlgorithmMod.CheckPoint(newCheckP,path,chk_name)
    try:
        pop, log = AlgorithmMod.eaSimpleModWithElitism(path,newCheckP,pop, toolbox, 0.80, 0.20, Ngen, run, 1,stats=mstats,
                                        halloffame=hof,checkpoint=CheckP,verbose=True)
    except:
        traceback.print_exc()

    try:
        best = hof.items[0]
    except:
        input("Press Enter to continue...")
        os.kill(os.getpid(), signal.SIGTERM)

    bestFitness= best.fitness.values[0]
    #Extract statistics
    minFitnessValues, meanFitnessValues=log.chapters["fitness"].select("min","avg")
    #plot statistics
    sns.set_style("whitegrid")
    plt.plot(minFitnessValues,marker='*', color='red')
    plt.xlabel('Generation')
    plt.ylabel('Min Fitness')
    plt.title('Min Fitness over Generations')
    plt.savefig(path+'Statistics'+str(run)+'.eps')
    plt.savefig(path+'Statistics'+str(run)+'.png')
    plt.close()
    end = time.time()
    finaltime=end-start
    print("Multiprocessing with",CPU_count,"core(s) took",round(finaltime,2),"s")
    #Calculamos el tiempo transcurrido de la ejecucion del programa
    #Imprimimos en segundos
    if(finaltime<=60):
        sys.stdout.write("\nTiempo transcurrido: %f seg\n"%(finaltime))
    #Imprimimos en minutos
    elif(finaltime>60)and(finaltime<3600):
        sys.stdout.write("\nTiempo transcurrido: %f min\n"%(finaltime/60))
    #Imprimimos en horas
    elif(finaltime>3600)and(finaltime<86400):
        sys.stdout.write("\nTiempo transcurrido: %f horas\n"%(finaltime/3600))
    #Imprimimos en dias
    elif(finaltime>86400)and(finaltime<2678000):
        sys.stdout.write("\nTiempo transcurrido: %f dias\n"%(finaltime/86400))
    #Finalizamos el programa
    input("Press Enter to continue...")
    os.kill(os.getpid(), signal.SIGTERM)
if __name__=="__main__":
    MP.freeze_support()
    #Directorio donde se almacenaran todos los datos
    pathA =str('Results/')
    pathB =str('Corrida'+str(N)+'/')
    path=pathA+pathB
    #Instrucciones para crear el directorio
    try:
        #Se intenta crear el directorio
        #Si no hay ninguno de nombre repetido se crea una nueva carpeta
        os.mkdir(pathA)
        #imprimimos un texto al usuario si se creo la carpeta
        sys.stdout.write("\nCreacion completa del directorio: %s\n"%pathA)
        sys.stdout.flush()
        #Accedemos a la simulacion y parametros evolutivas
    except OSError:
        gc.collect()
        #Si la carpeta ya existe se muestra al usuario un texto avisando que la carpeta sera sobreescribida
        sys.stdout.flush()
    try:
        #Se intenta crear el directorio
        #Si no hay ninguno de nombre repetido se crea una nueva carpeta
        os.mkdir(path)
        #imprimimos un texto al usuario si se creo la carpeta
        sys.stdout.write("\nCreacion completa del directorio: %s\n"%path)
        sys.stdout.flush()
        #Accedemos a la simulacion y parametros evolutivas
    except OSError:
        gc.collect()
        #Si la carpeta ya existe se muestra al usuario un texto avisando que la carpeta sera sobreescribida
        sys.stdout.flush()
    Main(N,path)
"""
Create the fitness class
:param ind => Individual from DEAP toolbox
:param td => Desired time
:param Npros => Number of initial conditions
"""



Creacion completa del directorio: Results/

Creacion completa del directorio: Results/Corrida2/

Creando nueva evolucion


# Programa Principal

Created on Fri Nov 17 21:00:33 2023
Nota: En caso de no existir dependencia entre arboles,
se toma el primer valor de la lista la cual es el arbol
que se evalua.
Ejemplo: [Ind4, Ind3, Ind2, Ind1]
Se evaluara el Ind4 y ese sera el que se deba usar, solo
en caso de no existir dependencia entre eslabones
@author: serdg